In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [27]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [28]:
EPOCHS = 50
BATCH_SIZE = 64

In [29]:
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                  train=True,
                  download=True,
                  transform=transforms.Compose([
                      transforms.RandomHorizontalFlip(),   # 랜덤 반전으로 데이터 늘리기 => overfitting 줄임
                      transforms.ToTensor(),
                      transforms.Normalize((0.1307,),(0.3081,))
                  ])),
    batch_size = BATCH_SIZE,
    shuffle = True
)
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                  train=False,
                  download=True,
                  transform=transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize((0.1307,),(0.3081,))
                  ])),
    batch_size = BATCH_SIZE,
    shuffle = True
)

In [30]:
class Net(nn.Module):
    def __init__(self, dropout_p=0.2):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784,256)
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,10)
        self.dropout_p = dropout_p
        
    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        
        # dropout => overfitting 줄임
        x = F.dropout(x, training=self.training, p=self.dropout_p) 
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training, p=self.dropout_p)
        x = self.fc3(x)
        
        return x

In [31]:
model = Net(dropout_p=0.2).to(DEVICE)

In [32]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [33]:
def train(model, train_loader, optimizer):
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [34]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    
    return test_loss, test_accuracy

In [35]:
for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))

[1] Test Loss: 0.6433, Accuracy: 77.08%
[2] Test Loss: 0.5450, Accuracy: 80.20%
[3] Test Loss: 0.4873, Accuracy: 82.19%
[4] Test Loss: 0.4581, Accuracy: 83.58%
[5] Test Loss: 0.4406, Accuracy: 83.98%
[6] Test Loss: 0.4318, Accuracy: 84.32%
[7] Test Loss: 0.4114, Accuracy: 85.14%
[8] Test Loss: 0.4049, Accuracy: 85.29%
[9] Test Loss: 0.3988, Accuracy: 85.44%
[10] Test Loss: 0.3867, Accuracy: 85.92%
[11] Test Loss: 0.3903, Accuracy: 85.66%
[12] Test Loss: 0.3765, Accuracy: 86.44%
[13] Test Loss: 0.3791, Accuracy: 86.16%
[14] Test Loss: 0.3701, Accuracy: 86.48%
[15] Test Loss: 0.3576, Accuracy: 87.12%
[16] Test Loss: 0.3616, Accuracy: 86.81%
[17] Test Loss: 0.3537, Accuracy: 87.17%
[18] Test Loss: 0.3639, Accuracy: 86.72%
[19] Test Loss: 0.3485, Accuracy: 87.34%
[20] Test Loss: 0.3424, Accuracy: 87.61%
[21] Test Loss: 0.3643, Accuracy: 86.68%
[22] Test Loss: 0.3439, Accuracy: 87.43%
[23] Test Loss: 0.3389, Accuracy: 87.64%
[24] Test Loss: 0.3415, Accuracy: 87.68%
[25] Test Loss: 0.3337, A